In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

In [3]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [4]:
train_x = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)

train_x, train_y = sm.fit_resample(train_x, train_y)

rfc = RandomForestClassifier(n_estimators=100,
                              max_features=4, 
                              n_jobs=-1, oob_score=True)

gbc = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1)
lgb = lgb.LGBMClassifier(learning_rate= 1, n_estimators = 100)
svc = SVC(gamma='auto', probability=True)
lrb = LogisticRegression(random_state=0)
xgb = XGBClassifier()

print("First")
eclf1 = VotingClassifier(estimators=[('xgb', xgb), ('gbc', gbc), ('lgb', lgb)], voting='soft')
eclf1.fit(train_x, train_y)

print("Second")
eclf2 = VotingClassifier(estimators=[('xgb', xgb), ('svc', svc), ('lgb', lgb)], voting='soft')
eclf2.fit(train_x, train_y)

print("Third")
eclf3 = VotingClassifier(estimators=[('xgb', xgb), ('gbc', gbc), ('svc', svc)], voting='soft')
eclf3.fit(train_x, train_y)

print("Forth")
eclf4 = VotingClassifier(estimators=[('lgb', lgb), ('svc', svc), ('gbc', gbc)], voting='soft')
eclf4.fit(train_x, train_y)

# forest = RandomForestClassifier(n_estimators=100, n_jobs=-1)
# forest.fit(train_x, train_y)

Using TensorFlow backend.


First


In [ ]:
y_pred1 = eclf1.predict_proba(X_test)
y_pred2 = eclf2.predict_proba(X_test)
y_pred3 = eclf3.predict_proba(X_test)
y_pred4 = eclf4.predict_proba(X_test)

y_pred = (y_pred1*y_pred2*y_pred3*y_pred4)**(1/4)

In [ ]:
# y_pred = forest.predict_proba(test_x)

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)

In [ ]:
train_y.value_counts()